In [21]:
import os
import pandas as pd
from numpy.random import rand

path = '/Users/nikkivanhandel/Desktop/L0001.csv'

The following data set is from the NIST Additive Manufacturing Metrology Testbed. The given file is a single layer for their machine

https://www.nist.gov/publications/process-monitoring-dataset-additive-manufacturing-metrology-testbed-ammt-overhang-part

In [16]:
# Channel names (x position, yposition, power, trigger (laser on or off))
df = pd.read_csv(path, names=['X', 'Y', 'P', 'T']) 
df.drop('T', axis=1, inplace=True) # Trigger command isn't imporant 

In the original file, absolute time is not given. Rather, each row corresponds to a command given at 100 kHz; each row is 1 microsecond apart. While the data files for this build are available, its simpler to just add noise to this to simulate a recorded data. To use this for the example, we will downsample to data to 100 Hz and add noise +/- 5 volts.

In [22]:
# Downsample from 100,000 Hz to 100 Hz, each time incrememnt is 0.01s apart
downsampled = df.loc[::1000].reset_index(drop=True)
rand(10)

array([0.27119223, 0.75710014, 0.38668946, 0.87160551, 0.91478004,
       0.25917804, 0.22454185, 0.26519482, 0.28486714, 0.93462062])